# DeepEyeNet-Image Captioning with key word reinforced
## Evaluation Part

In [10]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import glob
import json
import os
import re
import pickle
from tqdm import tqdm
from collections import defaultdict
import nltk
import h5py

from keras.optimizers import Adam, RMSprop
from keras.models import load_model

from model import CaptionModel, KeywordModel

# Load Data

In [11]:
def load_json(file):
    with open(file,'r') as f:
        return json.load(f)
def save_json(data, file):
    with open(file,'w') as f:
        json.dump(data, f)
def load_pickle(file):
    with open(file,'rb') as f:
        return pickle.load(f)
def save_pickle(data, file):
    with open(file,'wb') as f:
        pickle.dump(data, f)

In [12]:
# train_imgs, val_imgs, test_imgs
train_imgs = load_pickle('./data/data_path/train_imgs.pkl')
val_imgs = load_pickle('./data/data_path/val_imgs.pkl')
test_imgs = load_pickle('./data/data_path/test_imgs.pkl')

# Important features
results = load_pickle('./data/data_path/results.pkl')

In [13]:
# MODEL (No keywords)
# Parameters
vocab_size = len(results['word2id'])
max_len = max([len(x) for x in results['cap_toks'].values()])
embedding_size = 300

# load model
model_obj = CaptionModel(embedding_size, vocab_size, max_len, results['word2id'], results['id2word'])
final_model = model_obj.forward()
final_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
final_model = load_model('./model/model_vgg_new.h5')

In [14]:
# MODEL (with keywords)
# Parameters renewed
new_vocab_size = len(results['word2id_keys'])
key_max_len = max([len(x) for x in results['keywords_ids'].values()])

model_obj_k = KeywordModel(key_max_len, results['keywords_ids'], embedding_size, new_vocab_size,
                           vocab_size, max_len, results['word2id'], results['id2word'])
final_model_k = model_obj_k.forward()
final_model_k.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
final_model_k = load_model('./model/model_vgg_new_keywords.h5')

# Evaluation